In [1]:
import sclblonnx as so
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import onnx
from onnx import AttributeProto, TensorProto, GraphProto

## Construction modèle Skmean

In [2]:
data = pd.read_excel("data.xlsx",index_col=0)
int_feat = [f for f in data.columns if data.dtypes[f]==np.int64]
data[int_feat] = data[int_feat].astype(float)

In [3]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

num_feat = [f for f in data.columns if data.dtypes[f]==np.float64]
cat_feat = [f for f in data.columns if data.dtypes[f]==object]


num_transformer   = Pipeline(steps=[('minmax', MinMaxScaler())])
cat_transformer   = Pipeline(steps=[('cat', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_transformer , num_feat),
            ('cat', cat_transformer , cat_feat),
        ])

model_detection = Pipeline(steps=[('prepro',preprocessor),('model',KMeans(n_clusters=10))])

In [4]:
model_detection.fit(data)

C:\Users\33623\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('minmax',
                                                                   MinMaxScaler())]),
                                                  ['EPAI_VISEE', 'LARG_VISEE',
                                                   'LONG_MES', 'T_B_PR_MO',
                                                   'T_ext', 'TB_MT_MO',
                                                   'VIT_LIGN_MO']),
                                                 ('cat',
                                                  Pipeline(steps=[('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['M1_ACIER',
                                                   'OF_RVTTYPE'])])),
                ('model', KMeans(n_clusters=10))])

In [6]:
2*model_detection.predict(data)

array([14, 14, 14, ..., 10, 18, 18])

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin

class multiply_by(BaseEstimator,TransformerMixin):
    def __init__(self,factor=2.0):
        self.factor = factor
        
    def multiply_input(self,X,y=None):
        return 2*X
    
    def predict(self,X,y=None):
        y = 2*X
        return y
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        print(X.shape)
        X = pd.Series(X).copy()
        return X.apply(self.multiply_input)

In [35]:
model_detection2 = Pipeline(steps=[('prepro',preprocessor),('model',KMeans(n_clusters=3))])



In [37]:
model_detection2.transform(data)

array([[2.083382  , 1.16246526, 1.79816616],
       [2.03023719, 1.07407345, 1.72800916],
       [2.03158513, 1.073865  , 1.72720865],
       ...,
       [1.47818538, 1.8268057 , 1.04295427],
       [1.44524659, 1.7619612 , 1.06502367],
       [1.43930415, 1.76241562, 1.06807213]])

In [36]:
model_detection2.fit(data)

C:\Users\33623\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('minmax',
                                                                   MinMaxScaler())]),
                                                  ['EPAI_VISEE', 'LARG_VISEE',
                                                   'LONG_MES', 'T_B_PR_MO',
                                                   'T_ext', 'TB_MT_MO',
                                                   'VIT_LIGN_MO']),
                                                 ('cat',
                                                  Pipeline(steps=[('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['M1_ACIER',
                                                   'OF_RVTTYPE'])])),
                ('model', KMeans(n_clusters=3))])

In [16]:
pred = model_detection2.predict(data)

In [17]:
pred.shape

(10895, 10)

In [4]:
model_detection.fit(data)

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, StringTensorType,Int64TensorType



inputs = []
for k, v in zip(data.columns, data.dtypes):
    if v == 'int64':
        t = Int64TensorType([None, 1])
    elif v == 'float64':
        t = FloatTensorType([None, 1])
    else:
        t = StringTensorType([None, 1])
    inputs.append((k, t))

output = [('target',FloatTensorType([None, 1]))]

model_onnx = convert_sklearn(model_detection, initial_types=inputs,target_opset=12)

C:\Users\33623\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [6]:
# The serialization
with open("kmeans.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

: 

In [5]:
model_onnx

ir_version: 7
producer_name: "skl2onnx"
producer_version: "1.14.0"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "EPAI_VISEE"
    input: "LARG_VISEE"
    input: "LONG_MES"
    input: "T_B_PR_MO"
    input: "T_ext"
    input: "TB_MT_MO"
    input: "VIT_LIGN_MO"
    output: "merged_columns"
    name: "Concat"
    op_type: "Concat"
    attribute {
      name: "axis"
      i: 1
      type: INT
    }
    domain: ""
  }
  node {
    input: "M1_ACIER"
    output: "M1_ACIERout"
    name: "OneHotEncoder"
    op_type: "OneHotEncoder"
    attribute {
      name: "cats_strings"
      strings: "AB"
      strings: "AS"
      strings: "BT"
      strings: "CC"
      strings: "CE"
      strings: "DC"
      strings: "DM"
      strings: "DT"
      strings: "EF"
      strings: "HB"
      strings: "IB"
      strings: "ID"
      strings: "IF"
      strings: "IL"
      strings: "IU"
      strings: "IX"
      strings: "JG"
      strings: "LI"
      strings: "LT"
      strings: 

In [27]:
list_acier = list(data['M1_ACIER'].unique())

In [39]:
list_acier = np.asarray(list_acier)

In [44]:
list_acier.dtype

dtype('<U2')

In [72]:
help(onnx.helper.make_tensor)

Help on function make_tensor in module onnx.helper:

make_tensor(name: str, data_type: int, dims: Sequence[int], vals: Any, raw: bool = False) -> onnx.onnx_ml_pb2.TensorProto
    Make a TensorProto with specified arguments.  If raw is False, this
    function will choose the corresponding proto field to store the
    values based on data_type. If raw is True, use "raw_data" proto
    field to store the values, and values should be of type bytes in
    this case.
    
    Arguments:
        name (string): tensor name
        data_type (int): a value such as onnx.TensorProto.FLOAT
        dims (List[int]): shape
        vals: values
        raw (bool): if True, vals contains the seralized content of the tensor,
            otherwise, vals should be a list of values of the type defined by *data_type*
    
    Returns:
        TensorProto



In [71]:
onnx.helper.make_tensor('X', TensorProto.STRING,dims=(2,),vals= ['toto','tit'])

TypeError: 'toto' has type str, but expected one of: bytes

In [52]:
TensorProto.STRING

8

In [45]:
onnx.mapping.NP_TYPE_TO_TENSOR_TYPE[list_acier.dtype]

KeyError: dtype('<U2')

In [43]:
list_acier.flatten().astype(str)

array(['CE', 'IX', 'ID', 'IF', 'PR', 'SO', 'LT', 'LX', 'AS', 'SD', 'IB',
       'SF', 'SB', 'UC', 'DM', 'DT', 'JG', 'EF', 'AB', 'IL', 'HB', 'PP',
       'DC', 'LU', 'LI', 'UU', 'BT', 'IU', 'CC'], dtype='<U2')

In [57]:
value = onnx.helper.make_tensor_value_info('categorieacier',vals=list_acier)

TypeError: make_tensor_value_info() got an unexpected keyword argument 'vals'

In [59]:
g = so.empty_graph()

In [60]:
type(g)

onnx.onnx_ml_pb2.GraphProto

In [61]:
type(n1)

onnx.onnx_ml_pb2.NodeProto

In [65]:
# Add a node to the graph.
n1 = so.node('Add', inputs=['x1', 'x2'], outputs=['sum'])
g = so.add_node(g, n1) 
# Add inputs:
g = so.add_input(g, 'x1', "FLOAT", [1])
g = so.add_input(g, 'x2', "FLOAT", [1]) 
# And, add an output.
g = so.add_output(g, 'sum', "FLOAT", [1])

In [67]:
g = so.add_constant(g, "typeac", list_acier, "STRING")

ERROR: Data string not found. Use `list_data_types()` to list all supported data strings.


In [69]:
so.list_data_types()

{
  "FLOAT": 1,
  "UINT8": 2,
  "INT8": 3,
  "UINT16": 4,
  "INT16": 5,
  "INT32": 6,
  "INT64": 7,
  "BOOL": 9,
  "FLOAT16": 10,
  "DOUBLE": 11,
  "UINT32": 12,
  "UINT64": 13,
  "COMPLEX64": 14,
  "COMPLEX128": 15
}
Note: STRINGS are not supported at this time.


True

In [68]:
help(so.add_constant)

Help on function add_constant in module sclblonnx.constant:

add_constant(graph: onnx.onnx_ml_pb2.GraphProto, name: str, value: <built-in function array>, data_type: str, **kwargs)
    Create and add a constant node to an existing graph.
    
    Note: use add_node() if you want to add an existing constant node to an existing graph
    
    Args:
        graph: A graph, onnx.onnx_ml_pb2.GraphProto.
        name: Name of the (output value of the) constant node to determine the graph topology
        value: Values of the node (as a np.array)
        data_type: Data type of the node
    
    Returns:
        The extended graph.



In [63]:
import numpy as np

# First, let's clean the graph (not really necessary here) 
g = so.clean(g) 
# Next, lets see if it passes all checks:
so.check(g)
# Display the graph
display(g) 
# Evaluate the graph:
example = {
    "x1": np.array([1.2]).astype(np.float32), 
    "x2": np.array([2.5]).astype(np.float32)
   }
result = so.run(g,
    inputs=example,                
    outputs=["sum"])
print(result)

The graph was successfully cleaned.
Running Scailable specific checks for WASM conversion. 
Use _sclbl_check=False to turn off
Your graph was successfully checked.


node {
  input: "x1"
  input: "x2"
  output: "sum"
  name: "sclbl-onnx-node4"
  op_type: "Add"
}
name: "sclblgraph"
input {
  name: "x1"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
      }
    }
  }
}
input {
  name: "x2"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
      }
    }
  }
}
output {
  name: "sum"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
      }
    }
  }
}

[array([3.7], dtype=float32)]


In [64]:
so.graph_to_file(g, "filename.onnx")

True